# Molecular Simulation

Using Lennard-Jones potential

In [31]:
import numpy as np
import pygame

In [94]:
MASS = 1.
SIGMA = 1.
EPSILON = 1.

### Code stuff

In [95]:
# Force derived from Lennard-Jones potential, F = -gradV
def lennard_jones_force(X):
	'''
	X = [dpx, dpy, vx, vy], an ndarray
	dpx, dpy: difference between position of molecule that
	will be updated and molecule that is creating the force
	vx, vy: velocity of molecule that will be updated
	'''
	dpx, dpy, vx, vy = X
	Dpx = vx
	Dpy = vy
	drs = dpx**2 + dpy**2 + 5e-3 # Distance between particles squared
	c = 24 * EPSILON * (SIGMA ** 6) * (1 / MASS) * (drs ** (-4)) * (2 * (SIGMA ** 6) * (drs ** (-3)) - 1) # Insane constant
	Dvx = c * dpx
	Dvy = c * dpy
	return np.array([Dpx, Dpy, Dvx, Dvy])

In [96]:
def RK_method(F, X, h, a, b, c):
	dim = len(X)
	l = len(b)
	k = np.zeros((l, dim))

	k[0] = F(X)
	for i in range(1, l):
		x_i = X + h * np.dot(a[i-1][:i], k[:i])
		k[i] = F(x_i)
		
	return h * np.dot(b, k)

a = np.array(
    [[0.5, 0], [-1, 2]]
)
b = np.array([1/6, 4/6, 1/6])
c = np.array([0, 0.5, 1])

def RK3(F, X, h):
    return RK_method(F, X, h, a, b, c)

In [97]:
def euler(F, X, h):
	return h * F(X)

In [98]:
# Update particles according to the ODE that determines the system
def simulation_step(particles, h):
	updated_particles = np.copy(particles)
	for i, p1 in enumerate(particles):
		for p2 in particles:
			if p1 is not p2:
				dpx, dpy = p1[:2] - p2[:2]
				vx, vy = p1[2:]
				X = np.array([dpx, dpy, vx, vy])
				# X = np.concatenate((p1[:2] - p2[:2]), p1[2:]) FIX THIS LATER I GUESS
				updated_particles[i] += RK3(lennard_jones_force, X, h)
		if updated_particles[i, 0] < 0 or updated_particles[i, 0] >= 1:
			updated_particles[i, 2] *= -1
		if updated_particles[i, 1] < 0 or updated_particles[i, 1] >= 1:
			updated_particles[i, 3] *= -1
	return updated_particles

In [99]:
# Initialize pygame display
def init_pygame_display(window_size=(500, 500)):
    pygame.init()
    screen = pygame.display.set_mode(window_size)
    pygame.display.set_caption("Position Simulation")
    return screen

# Function to display simulation using pygame
def run_simulation(particles, window_size=(500, 500), point_radius=5, speed=1e-2):
    screen = init_pygame_display(window_size)
    clock = pygame.time.Clock()
    running = True
    
    while running:
        # Check for Pygame events (such as quitting the simulation)
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
                pygame.quit()
                return  # Exit function
        
        # Update particles' positions with the simulation step
        particles = simulation_step(particles, speed)
        
        # Scale the particles' positions to fit within the window
        scaled_particles = particles[:, :2] * window_size[0]  # Assuming positions are in [0, 1]
        
        # Clear the screen with a white background
        screen.fill((255, 255, 255))

        # Draw each particle as a circle
        for pos in scaled_particles:
            x, y = pos
            pygame.draw.circle(screen, (0, 0, 255), (int(x), int(y)), point_radius)

        # Update the display
        pygame.display.flip()

        # Cap the frame rate at 30 FPS
        clock.tick(30)

### Actual sim

In [100]:
n_particles = 50

# Each particle has an px, py position and vx, vy velocity
particles = np.random.uniform(size=(n_particles, 4))
print(particles[:5])

[[0.41515085 0.50043062 0.71786479 0.93790769]
 [0.38000727 0.08146088 0.95627725 0.86824427]
 [0.53605519 0.79075687 0.06439254 0.99627775]
 [0.9437292  0.58252018 0.1295168  0.10862996]
 [0.05681138 0.25237619 0.53513746 0.01602055]]


In [101]:
run_simulation(particles, window_size=(500, 500), point_radius=5., speed=1e-2)

TypeError: center argument must be a pair of numbers